In [ ]:
# default_exp secret
# hide

_FNAME='secret'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os

TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
HERE = os.path.abspath('')

In [ ]:
#export
import logging

import boto3

logger=logging.getLogger()
CLIENT = boto3.client('secretsmanager')

In [ ]:
#export 
from scylla import run_container, set_port_name
import localstack_client.session

def run_localstack(version="latest", **kwargs):
    env = {'SERVICES':'secretsmanager'}
    additional_kwargs = {'environment': env}
    additional_kwargs.update(kwargs)
    #changing the port of localstack is hard and involves tinkering with the EDGE_PORT environment variable
    #just leave it be
    ports = {4566:4566}
    image = 'localstack/localstack:{}'.format(version)
    ls_container = run_container(image, ports=ports, block_until_port_available=4566,
                                   **additional_kwargs)
    ls_container = set_port_name(ls_container, 4566, 'entry')
    return ls_container    

def make_local_client():
    session = localstack_client.session.Session()
    return session.client('secretsmanager')

In [ ]:
CONTAINER_NAME = 'localstack-vera-secret'
ls_container = run_localstack(container_name=CONTAINER_NAME)

In [ ]:
#export
def get(name, client=None, fail_safely=False, **kwargs):
    '''
    Retrieves the secret with the given name
    '''
    client = client or CLIENT
    try:
        resp = client.get_secret_value(SecretId=name, **kwargs)
    except client.exceptions.ResourceNotFoundException:
        if fail_safely:
            return None
        raise    
    str_value = resp.get('SecretString')
    byte_value = resp.get('SecretBytes')
    return str_value or byte_value

def create(name, value, client=None, update_if_exists=False, **kwargs):
    '''
    Create a new secret with Name=name
    If you want SecretBytes instead of SecretString
    set value=None and specify SecretBytes in kwargs
    '''
    client = client or CLIENT
    if value is not None:
        kwargs['SecretString'] = value
    try:
        resp = client.create_secret(Name=name, **kwargs)
        ok = resp['ResponseMetadata']['HTTPStatusCode']==200        
    except client.exceptions.ResourceExistsException:
        if not update_if_exists:
            raise
        ok, resp = update(name=name, value=value, client=client, **kwargs)
    return ok, resp

def update(name, value, client=None, **kwargs):
    '''
    Update the value of a secret with SecretId=name
    '''
    client = client or CLIENT
    if value is not None:
        kwargs['SecretString'] = value
    
    resp = client.put_secret_value(SecretId=name, **kwargs)    
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

def delete(name, client=None, **kwargs):
    '''
    Schedule the secret with SecretId=name for deletion
    '''
    client = client or CLIENT
    resp = client.delete_secret(SecretId=name, **kwargs)    
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

In [ ]:
import uuid
def LIVE_test():
    client = make_local_client()
    
    secret_name = 'unittest/{}'.format(uuid.uuid4())

    #get an secret before its created
    with TESTCASE.assertRaises(client.exceptions.ResourceNotFoundException):
        get(secret_name, client=client)
    TESTCASE.assertIsNone(get(secret_name, client=client, fail_safely=True))
    ok, resp = create(secret_name, 'testvalue', client=client)
    #print(ok, resp)
    TESTCASE.assertTrue(ok)

    TESTCASE.assertEqual( get(secret_name, client=client), 'testvalue')

    ok, resp = update(secret_name, 'newvalue', client=client)
    #print(ok, resp)
    TESTCASE.assertTrue(ok)
    TESTCASE.assertEqual( get(secret_name, client=client), 'newvalue')
    
    ok, resp = create(secret_name, 'newervalue', client=client, update_if_exists=True)
    #print(ok, resp)
    TESTCASE.assertTrue(ok)
    TESTCASE.assertEqual( get(secret_name, client=client), 'newervalue')
    
    with TESTCASE.assertRaises(client.exceptions.ResourceExistsException):
        create(secret_name, 'you will never see this', client=client, update_if_exists=False)
    
    ok, resp = delete(secret_name, client=client)
    #print(ok, resp)

    with TESTCASE.assertRaises(CLIENT.exceptions.InvalidRequestException):
        get(secret_name, client=client)
        
LIVE_test()

In [ ]:
#export
class LazyLoader:
    def __init__(self, client=None):
        self.client = client
        self._secrets = {}
        
    def _get(self, *args, **kwargs):
        return get(*args, **kwargs)
        
    def get(self, name, *args, **kwargs):
        try:
            return self._secrets[name]
        except KeyError:
            secret_value = self._get(name, *args, client=self.client, **kwargs)
            if secret_value is not None:
                self._secrets[name] = secret_value
            return secret_value
    

In [ ]:
def test_lazy():
    ll = LazyLoader()
    ll._get = mock.MagicMock()
    ll._get.return_value = 'not a real value'

    secret_name = 'unittest/fake_secret'
    ll.get(secret_name)
    ll.get(secret_name)
    ll.get(secret_name)
    
    ll._get.assert_called_once()
    
test_lazy()

In [ ]:
ls_container.stop()

In [ ]:
notebook2script(_nbpath)